In [1]:
import pandas as pd
import pickle
import os
from sklearn.preprocessing import StandardScaler

# 加载模型
filename = 'loan_approval_model.pkl'
model_dir = 'model'  # 模型存储目录
model_path = os.path.join(model_dir, filename)  # 模型存储路径
loaded_model = pickle.load(open(model_path, 'rb'))

print("模型已加载")

模型已加载


In [2]:
# 准备新数据
new_data = pd.DataFrame({
    'Income': [61853],  # 替换为实际收入
    'Credit_Score': [732],  # 替换为实际信用评分
    'Loan_Amount': [19210],  # 替换为实际贷款金额
    'DTI_Ratio': [38],  # 替换为实际 DTI 比率
    'Employment_Status': [1]  # 替换为实际就业状态 (1: employed, 0: unemployed)
})

# 加载 scaler 和 特征列名
scaler_path = os.path.join(model_dir, 'scaler.pkl')
scaler, feature_names = pickle.load(open(scaler_path, 'rb')) # 加载 scaler 和 列名
print("Scaler已加载")

# 确保新数据的列名与训练数据一致
# 检查 new_data 是否包含所有需要的特征
for feature in feature_names:
    if feature not in new_data.columns:
        print(f"错误：新数据缺少特征 {feature}，请检查数据")
        exit()  # 退出程序，因为缺少特征无法进行预测

new_data = new_data[feature_names]  # 按照模型训练时的特征顺序进行选择

# 对新数据进行缩放
new_data_scaled = scaler.transform(new_data)

# 重新赋值列名
new_data_scaled = pd.DataFrame(new_data_scaled, columns=feature_names)

# 使用加载的模型进行预测
new_predictions = loaded_model.predict(new_data_scaled)

# 打印预测结果
if new_predictions[0] == 1:
    print("预测结果：贷款已批准")
else:
    print("预测结果：贷款已拒绝")

Scaler已加载
预测结果：贷款已批准


In [3]:
# 基于用户输入申请的文本匹配
import pandas as pd
import os
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
df = pd.read_csv("data/loan_data_with_sentiment_analysis.csv")

embeddings_zp = ZhipuAIEmbeddings(
    model="embedding-3",
    api_key=os.getenv("ZHIPUAI_API_KEY"),
)

db_load = Chroma(
    persist_directory="db/vectorstore_loan",
    embedding_function=embeddings_zp
)

# 构造函数来实现 多结果的现实
def retrieve_similar_applications(
        query: str,
        top_k: int = 5,
) -> pd.DataFrame:
    recs = db_load.similarity_search(query, k = 10)

    text_list = []

    for i in range(0, len(recs)):
        text_list += [int(recs[i].page_content.strip('"').split()[0])]

    return df[df["id"].isin(text_list)]

In [5]:
retrieve_similar_applications("I want buy a car")

,Text,Income,Credit_Score,Loan_Amount,DTI_Ratio,Employment_Status,Approval,id,tagged_text,predicted_category,anger,disgust,fear,joy,sadness,surprise,neutral
156,I want to buy a car for my rideshare business ...,109096,819,1336,20.63,unemployed,Rejected,156,156 I want to buy a car for my rideshare busin...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
1119,I want to buy a car for my rideshare business ...,72101,633,52050,27.85,employed,Rejected,1119,1119 I want to buy a car for my rideshare busi...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
5209,I want to buy a car for my rideshare business ...,109501,772,68074,13.54,unemployed,Rejected,5209,5209 I want to buy a car for my rideshare busi...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
7308,I want to buy a car for my rideshare business ...,26926,789,19470,174.73,unemployed,Rejected,7308,7308 I want to buy a car for my rideshare busi...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
7658,I want to buy a car for my rideshare business ...,95478,366,15640,45.49,unemployed,Rejected,7658,7658 I want to buy a car for my rideshare busi...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
8301,I want to buy a car for my rideshare business ...,112096,379,8793,12.13,unemployed,Rejected,8301,8301 I want to buy a car for my rideshare busi...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
9080,I want to buy a car for my rideshare business ...,109823,367,6510,38.61,employed,Rejected,9080,9080 I want to buy a car for my rideshare busi...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
12313,I want to buy a car for my rideshare business ...,80388,300,2080,45.40,unemployed,Rejected,12313,12313 I want to buy a car for my rideshare bus...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
12347,I want to buy a car for my rideshare business ...,22381,375,3677,138.95,unemployed,Rejected,12347,12347 I want to buy a car for my rideshare bus...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
20190,I want to buy a car for my rideshare business ...,41850,478,28477,111.92,unemployed,Rejected,20190,20190 I want to buy a car for my rideshare bus...,Car Purchase,0.015677,0.001804,0.019941,0.103844,0.845852,0.005595,0.007287
